In [1]:
import numpy as np
import random
import math


In [2]:
ROW_COUNT = 6
COLUMN_COUNT = 7

def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT), dtype=int)

In [3]:
def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r


In [4]:
def winning_move(board, piece):
    # Horizontal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if all(board[r, c+i] == piece for i in range(4)):
                return True
    # Vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if all(board[r+i, c] == piece for i in range(4)):
                return True
    # Diagonal /
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if all(board[r-i, c+i] == piece for i in range(4)):
                return True
    # Diagonal \
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if all(board[r+i, c+i] == piece for i in range(4)):
                return True
    return False


In [5]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = 1 if piece == 2 else 2
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(0) == 1:
        score += 10
    elif window.count(piece) == 2 and window.count(0) == 2:
        score += 5
    if window.count(opp_piece) == 3 and window.count(0) == 1:
        score -= 80
    return score

def score_position(board, piece):
    score = 0
    # Center column preference
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    score += center_array.count(piece) * 6

    # Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT-3):
            window = row_array[c:c+4]
            score += evaluate_window(window, piece)

    # Vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # Diagonal \
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Diagonal /
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+3-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score


In [6]:
def get_valid_locations(board):
    return [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]

def is_terminal_node(board):
    return winning_move(board, 1) or winning_move(board, 2) or len(get_valid_locations(board)) == 0

def minimax(board, depth, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, 2):
                return (None, 100000000000000)
            elif winning_move(board, 1):
                return (None, -10000000000000)
            else:  # Draw
                return (None, 0)
        else:
            return (None, score_position(board, 2))
    
    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, row, col, 2)
            new_score = minimax(temp_board, depth-1, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
        return best_col, value
    else:
        value = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, row, col, 1)
            new_score = minimax(temp_board, depth-1, True)[1]
            if new_score < value:
                value = new_score
                best_col = col

        return best_col, value


In [7]:
def print_board(board):
    symbol_map = {0:' ', 1:'H', 2:'C'}
    for i in range(ROW_COUNT):
        print('| ' + ' '.join(f'{symbol_map[board[i][j]]:2}|' for j in range(COLUMN_COUNT)))
    print(' ' + ' '.join(f'{j:2} ' for j in range(COLUMN_COUNT)))


In [11]:
from IPython.display import clear_output
game_over = False
turn = 0  # 0 = Human, 1 = Computer
board = create_board()

while not game_over:
    if turn == 0:
        col = int(input("Human move (0–6): "))
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, 1)
            if winning_move(board, 1):
                print("Human wins!")
                game_over = True
    else:
        col, minimax_score = minimax(board, 3, True)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, 2)
            if winning_move(board, 2):
                print("Computer wins!")
                game_over = True

    print_board(np.flip(board, 0))  # Flip for visual clarity
    #clearing the message of current loop and attempting to show a fresh message to user
    clear_output(wait=True)
    turn += 1
    turn %= 2


Computer wins!
|   |   | C | C |   | H |   |
|   |   | C | H |   | H |   |
|   |   | H | C |   | C |   |
|   |   | H | H | C | H |   |
|   |   | H | C | H | H |   |
| C | C | C | H | H | C | C |
  0   1   2   3   4   5   6 
